In [20]:
from datetime import datetime, timedelta
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from PIL import Image
import io
import numpy as np
# 
# set up oauth session
# client_id = 'sh-f1e886d6-8fb7-4c76-b844-eb933ae57311'
# client_secret = 'bFnAJq2nM0LUTf5TgdYfOFOs7AV0rS25'

client_id = 'sh-fc0feee4-2cee-4891-8af3-2c8bf49eac25'
client_secret = 'RDA2i3bDJNJ9smQde83TAMv5AnCD4MrP'

In [27]:
#  create a client
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# get the token
token = oauth.fetch_token(
    token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
    client_secret=client_secret
)

# define the Evalscript
evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: {
      bands: 3,
      sampleType: "AUTO",
    },
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}

"""

# set up the date range
start_date = datetime(2020, 12, 10)
end_date = datetime(2023, 6, 1)

# set up the request
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%dT00:00:00Z")
    next_date = current_date + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%dT00:00:00Z")

    request = {
        "input": {
            "bounds": {
                "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
                "geometry": {
                    "type": "Polygon",
                    # "coordinates": [
                    #     [
                    #         [103.849468, 1.184379],
                    #         [103.849468, 1.318071],
                    #         [104.100437, 1.318071],
                    #         [104.100437, 1.184379],
                    #         [103.849468, 1.184379]
                    #     ]
                    # ]
                    "coordinates": [[[103.868179, 1.234665], [103.91831775, 1.234665], [103.91831775, 1.272421], [103.868179, 1.272421], [103.868179, 1.234665]]] # 1-1
                    # "coordinates": [[[103.91831775, 1.234665], [103.9684565, 1.234665], [103.9684565, 1.272421], [103.91831775, 1.272421], [103.91831775, 1.234665]]] 1-2
                    # "coordinates": [[[103.868179, 1.272421], [103.91831775, 1.272421], [103.91831775, 1.310177], [103.868179, 1.310177], [103.868179, 1.272421]]] 1-3
                    # "coordinates": [[[103.91831775, 1.272421], [103.9684565, 1.272421], [103.9684565, 1.310177], [103.91831775, 1.310177], [103.91831775, 1.272421]]] 1-4
                    # "coordinates":[[[103.868179,1.234665],[103.9684565,1.234665],[103.9684565,1.310177],[103.868179,1.310177],[103.868179,1.234665]]] # part 1
                }
            },
            "data": [
                {
                    "type": "sentinel-2-l1c",
                    "dataFilter": {
                        "timeRange": {
                            "from": date_str,
                            "to": next_date_str
                        }
                    },
                }
            ],
        },
        "output": {
            "width": 2048,
            "height": 2048,
        },
        "evalscript": evalscript,
    }

    # set up the request
    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request)

    # check if the request was successful
    if response.ok:
        # read the image into a PIL image object
        image = Image.open(io.BytesIO(response.content))
        image_np = np.array(image)
        
        safe_date_str = date_str.replace(":", "_")
        
        
        count_over_threshold = np.sum(image_np > 200)
                
        if np.all(count_over_threshold > 0.5 * image_np.size):
            print(f"Image for {safe_date_str} is 1/2 white, not saved.")
        elif image_np.mean() > 10:  # check if the image is mostly dark
            file_name = f'./satellite_images/satellite_image_{safe_date_str}_1.png'
            image.save(file_name)
            print(f"Saved image for {safe_date_str}")
        else:
            print(f"Image for {safe_date_str} is mostly dark, not saved.")
            
    else:
        print(f"Failure {date_str}：", response.text)

    current_date = next_date


Image for 2020-12-10T00_00_00Z is mostly dark, not saved.
Saved image for 2020-12-11T00_00_00Z
Image for 2020-12-12T00_00_00Z is mostly dark, not saved.
Image for 2020-12-13T00_00_00Z is mostly dark, not saved.
Image for 2020-12-14T00_00_00Z is mostly dark, not saved.
Image for 2020-12-15T00_00_00Z is mostly dark, not saved.
Image for 2020-12-16T00_00_00Z is 1/2 white, not saved.
Image for 2020-12-17T00_00_00Z is mostly dark, not saved.
Image for 2020-12-18T00_00_00Z is mostly dark, not saved.
Image for 2020-12-19T00_00_00Z is mostly dark, not saved.
Image for 2020-12-20T00_00_00Z is mostly dark, not saved.
Saved image for 2020-12-21T00_00_00Z
Image for 2020-12-22T00_00_00Z is mostly dark, not saved.
Image for 2020-12-23T00_00_00Z is mostly dark, not saved.
Image for 2020-12-24T00_00_00Z is mostly dark, not saved.
Image for 2020-12-25T00_00_00Z is mostly dark, not saved.
Saved image for 2020-12-26T00_00_00Z
Image for 2020-12-27T00_00_00Z is mostly dark, not saved.
Image for 2020-12-28T